In [ ]:
pip install streamlit

In [ ]:
pip install -U langchain-community tavily-python

In [10]:
from crewai import Crew
import os
import requests
import json
from google.oauth2.credentials import Credentials as OAuth2Credentials
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_core.callbacks import BaseCallbackHandler
from typing import TYPE_CHECKING, Any, Dict
import streamlit as st


In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ["TAVILY_API_KEY"] = "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"
# set up the agent
tavily_tool = TavilySearchResults()

In [7]:
tavily_tool.run("How does exercise price determine for ESOP?")

[{'url': 'https://www.trica.co/equity/blog/how-to-calculate-exercise-price-in-esops/',
  'content': "To begin with, the exercise price in ESOP is used to determine the amount required to exercise the options and the tax ramifications of doing so. In return for the shares, the employee must pay the strike price multiplied by the number of vested options the employee desires to exercise. The difference between the stock's current Fair Market ..."},
 {'url': 'https://www.vega-equity.com/blog/how-to-calculate-exercise-price-in-esops',
  'content': "Now coming back to the exercise price in ESOPs, it is the price at which the shareholder can purchase the share options. Often termed the Strike price, the exercise price becomes active once the vesting period is completed. This price is set at the stock's fair market value at the time the stock options are granted."},
 {'url': 'https://www.fairvalueacademy.org/employee-share-options-esops/',
  'content': "The exercise multiple factors in early 

In [11]:
# Initialize the Gemini tool, this is just demo code
gemini_llm = GeminiTool(
    model_name=os.environ["MODEL_NAME"],
    auth_url=os.environ["AUTH_URL"]
) 

In [19]:
# Define the Research Agent
research_agent = Agent(
    role='Finance Researcher',
    goal='Discover and compile the latest finance news, market updates, and trends for {asset_class} in {market_data_feed}, focusing on {risk_models}.',
    verbose=True,
    memory=False,
    llm=gemini_llm,  
    backstory=(
        "As a dedicated finance researcher, you excel in finding and compiling "
        "relevant finance information that keeps finance professionals informed about the latest developments in their practice area. "
        "Please pass your search result in JSON format to your co-work, ensure the content to"
        "be enclosed in curly braces and use double quotes for keys and values."
    ),
    #tools=[search_tool],
    tools=[tavily_tool],
    allow_delegation=True
)

# Define the Summarization Agent
summarization_agent = Agent(
    role='Finance Summarizer',
    goal='Summarize the compiled finance information into easily digestible formats for {asset_class}.',
    verbose=True,
    memory=False,
    llm=gemini_llm,  
    backstory=(
        "As an expert in distilling complex finance information, you create concise and clear summaries "
        "that help finance professionals quickly grasp key updates and insights."
        "Please pass your search result in JSON format to your co-work, ensure the content to"
        "be enclosed in curly braces and use double quotes for keys and values."
    ),
    #tools=[search_tool],
    tools=[tavily_tool],
    allow_delegation=True
)

# Define the Research Task
research_task = Task(
    description=(
        "Gather the latest finance news, market updates, and emerging trends for {asset_class} in {market_data_feed}. "
        "Focus on finding information relevant to {risk_models}. Your final report should include a comprehensive summary "
        "of key updates and trends. Include refrences to the original sources (URLs) in the summary"
    ),
    expected_output=(
        "A detailed report containing the latest updates and trends relevant to the specified practice area, market_data_feed, and risk models."
        "Please pass your search result in JSON format to your co-work, ensure the content to"
        "be enclosed in curly braces and use double quotes for keys and values."),
    #tools=[search_tool],
    tools=[tavily_tool],
    agent=research_agent
)

# Define the Summarization Task
summarization_task = Task(
    description=(
        "Summarize the detailed report provided by the research task into key highlights. "
        "Ensure that the summary is concise and easy to understand, highlighting the most important developments."
    ),
    expected_output='A concise summary of the gathered information, highlighting key updates and insights. The summary must include refernces to orginal sources (URLs)',
    #tools=[search_tool],
    tools=[tavily_tool],
    agent=summarization_agent,
    async_execution=False,
    output_file='summary.txt'
)



In [20]:
# Form the Crew and kick it off
# Define user input prompts
asset_class = input("Please specify your asset class (e.g., Equities, Fixed Income, Real Estate): ")
market_data_feed = input("Please specify your market data feed source (e.g., Bloomberg, Reuters, Yahoo Finance):  ")
risk_models = input("Please specify the types of financial models you use (e.g., Value at Risk, Monte Carlo Simulation, Discounted Cash Flow): ") 
 
crew = Crew(
    agents=[research_agent, summarization_agent],
    tasks=[research_task, summarization_task],
    process=Process.sequential
)
 
# Example user inputs
inputs = {
    'asset_class': asset_class,
    'market_data_feed': market_data_feed,
    'risk_models': risk_models
}
 
result = crew.kickoff(inputs=inputs)
print(result)

Please specify your asset class (e.g., Equities, Fixed Income, Real Estate):  Equities
Please specify your market data feed source (e.g., Bloomberg, Reuters, Yahoo Finance):   Bloomberg
Please specify the types of financial models you use (e.g., Value at Risk, Monte Carlo Simulation, Discounted Cash Flow):  Value at Risk


2024-06-19 23:42:42,929 - 140233448482624 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Thought: I need to gather information on the latest finance news, market updates, and trends for Equities in Bloomberg, specifically related to Value at Risk. I should start by searching for recent developments and news articles on this topic. 
Action: tavily_search_results_json
Action Input: {"query": "Latest finance news and market updates for Equities in Bloomberg focusing on Value at Risk"}
Observation: <Response 200> {
"results": [
{
"title": "Rising Interest Rates Pose Challenges for Value at Risk (VaR) Models",
"url": "https://www.bloomberg.com/news/articles/2023-11-01/rising-interest-rates-pose-challenges-for-value-at-risk-var-models",
"snippet": "Traditional VaR models, widely used by financial institutions to estimate potential losses in their portfolios, are facing scrutiny as rising interest rates impact market volatility and correlations."
},
{
"title": "Equity Market Volatility Spikes as Inflation Concerns Linger",
"url": "https